## Математические основы машинного обучения
### Задание 1. 
Михаил Селюгин, Б05-876б

* Тип задачи: классификация
* Датасет: Breast Cancer Data Set
* Методы: KNN, перцептрон, логистическая регрессия

#### Анализ выборки

На первом этапе добавим необходимые библиотеки, загрузим выборку, рассмотрим предлагаемые признаки и проведем визуальный анализ данных.

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import matplotlib.pyplot as plt
from matplotlib.image import imread
from mpl_toolkits import mplot3d
from matplotlib import gridspec
import seaborn as sns
import pandas as pd
from tqdm.notebook import tqdm

from scipy.special import softmax
from scipy.spatial.distance import cdist
import numpy as np
import torch

from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC, SVR
from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import KFold, ParameterGrid, GridSearchCV, LeaveOneOut, RepeatedKFold
from sklearn.datasets import make_classification, make_regression
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier


from sklearn.decomposition import PCA

from sklearn.datasets import load_breast_cancer

from mlxtend.plotting import plot_decision_regions

from torchvision import datasets
from torchvision import transforms

ModuleNotFoundError: No module named 'mlxtend'

In [ ]:
data_cancer = load_breast_cancer()
X, y = data_cancer.data, data_cancer.target
#data_frame = load_breast_cancer(as_frame=True) не работает в старой версии sklearn



In [ ]:
X_frame = pd.DataFrame(X, columns=data_cancer.feature_names)
y_frame = pd.DataFrame(y, columns= ['class'])
data_frame = pd.concat([X_frame, y_frame], axis = 1, join = "inner")
print("Длина выборки = %d, количество признаков = %d" % (X.shape[0], X.shape[1]))
print("Классы:", data_cancer.target_names)
X_frame.head()

Таким образом, мы имеем задачу бинарной классификации. Класс задан нулем (злокачественная опухоль) или единицей (доброкачественная). У каждого из 569 объектов заданы 30 числовых признаков.

Проанализуем признаки с помощью корреляционных карт.

In [ ]:
plt.figure(figsize=(6,6))
heatmap = sns.heatmap(X_frame.corr())

In [ ]:
plt.figure(figsize=(8, 12))
heatmap = sns.heatmap(data_frame.corr()[['class']].sort_values(by='class', ascending=False), vmin=-1, vmax=1, annot=True, cmap='BrBG')
heatmap.set_title('Корреляция признаков с классом', fontdict={'fontsize':18}, pad=16);

Из построенных диаграм видно, что некоторые признаки скоррелированы, но глобальная тенденция не видна.

В то же время имеем блок признаков с низкой, близкой к -1, корреляцией с классом.

Заметим, что все 4 признака с наименьшей корреляцией, скоррелированы между собой достаточно хорошо.

In [ ]:
plt.figure(figsize=(6,6))
heatmap = sns.heatmap(data_frame[["worst concave points", "worst perimeter", "worst radius", "mean concave points"]].corr())

Поэтому в качестве отправной точки построим классифакцию по одному из этих признаков. 


In [ ]:
X_1cat = X_frame[['worst radius']]
X_1cat = np.hstack([X_1cat, np.ones([len(X_1cat), 1])])
X_train_1cat, X_test_1cat, y_train, y_test = train_test_split(X_1cat, y, test_size = 0.3)
model1 = LogisticRegression (random_state=0, max_iter=2000, fit_intercept=False)
_ = model1.fit(X_train_1cat, y_train)
print("Точность:", model1.score(X_test_1cat, y_test))

Получили довольно высокую точность. Проиллюстрируем с помощью диаграммы, где прямая обозначает полученный биас.



In [ ]:
x_bias = -model1.coef_[0][1]/model1.coef_[0][0]
data_for_plot=data_frame[['class', 'worst radius']]
pairplot1=sns.pairplot(data_for_plot,hue="class",height=4)
plt.axvline(x = x_bias)
plt.show()

Нормализуем признаки и перейдем к экспериментам.



In [ ]:
scaler = StandardScaler()
X=scaler.fit_transform(X)

### KNN

Построим метод k ближайших соседей для $k\in \{1,2,\ldots ,20\}$. Рассмотрим версию с равными весами всех соседей и модификацию с линейно убывающими весами. 

В качестве метрики для подбора гиперпараметров будем рассматривать точность. А сам подбор проведем с помощью GridSearchCV.



In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
def func(distance):
    ret = np.ones_like(distance)
    k = ret.shape[1]
    for i in range(k):
        ret[:, i] *= (k-i)/k
    return ret
  
def id(distance):
  return np.ones_like(distance)

In [ ]:
knn_search = GridSearchCV(KNeighborsClassifier(), {'n_neighbors': np.arange(1,30), 'weights' : [func, id]}, scoring="accuracy")
knn_search.fit(X, y)

In [ ]:
print("Best params:", knn_search.best_params_)
print("Accuracy:", knn_search.best_score_)


Получили, что наибольшую точность дает метод с одинаковыми весами для 9 соседей.

In [ ]:
KNN_best_model = KNeighborsClassifier(n_neighbors=9, weights= id)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
KNN_best_model.fit(X_train, y_train)
KNN_best_report = classification_report(KNN_best_model.predict(X_test), y_test)
print("KNN with 9 neighbours and equal weights\n", KNN_best_report)

### Перцептрон

Рассмотрим встроенную реализацию многослойного перцептнора с функцией активации RELU и с различными параметрами скрытых слоев: 1,2, 4 или 8 слоев размерности 8 или 64.

Оптимальные параметры подберем с помощью GridSearchCV.

In [ ]:
hidden_layers = []
for i in (1,2,4, 8):
  hidden_layers.append(tuple(np.full(i, 64)))
  hidden_layers.append(tuple(np.full(i, 8)))


In [ ]:
perceptron_search = GridSearchCV(MLPClassifier(), {'hidden_layer_sizes': hidden_layers, 'activation': ["relu"], 'random_state':[0], 'max_iter': [2000]}, scoring="accuracy")
perceptron_search.fit(X, y)

In [ ]:
print("Best params:", perceptron_search.best_params_)
print("Accuracy:", perceptron_search.best_score_)

Получили, что два скрытых слоя мощностью 64 дают наибольшую точность.

In [ ]:
Perceptron_best_model = MLPClassifier(hidden_layer_sizes= (64,64), activation = "relu", random_state = 0, max_iter = 2000)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
Perceptron_best_model.fit(X_train, y_train)
Perceptron_best_report = classification_report(Perceptron_best_model.predict(X_test), y_test)
print("Perceptron with (64, 64) hidden layers\n", Perceptron_best_report)

### Логистическая регрессия

Воспользуемся встроенной реализацией лог регрессии. Подбирать $\gamma$ будем с помощью Leave One Out.

In [ ]:
#1 min
loo = LeaveOneOut()
number_of_batch = loo.get_n_splits(X_train)
gammas = [1e-10, 1e-4, 1e-3, 1e-2, .1, 1., 10., 1e2, 1e3, 1e4, 1e10]

gamma_scores = dict()
for gamma in tqdm(gammas):
    list_of_scores = []
    for train_index, test_index in tqdm(loo.split(X_train), 
                                        total=number_of_batch, leave=False):
        x_train, x_test = X_train[train_index], X_train[test_index]
        y_lr_train, y_lr_test = y_train[train_index], y_train[test_index]

        model = LogisticRegression(penalty='l2', C=2/gamma, solver='saga',
                                   fit_intercept=False, random_state=0)
        model.fit(x_train, y_lr_train)
        list_of_scores.append(model.score(x_test, y_lr_test))
        
    gamma_scores[gamma] = np.mean(list_of_scores)


In [ ]:
best_gamma = sorted(gamma_scores, 
                    key=lambda x: gamma_scores[x], reverse=True)[0]
print("Оптимальное значение гамма:", best_gamma)
model = LogisticRegression(penalty='l2', C=2/best_gamma, fit_intercept=False, 
                           random_state=0, solver='saga')
model.fit(X_train, y_train)
print('Точность: {}'.format(model.score(X_test, y_test)))

Проверим также l1 регуляризацию. Т.к. солвер 'liblinear' работает медленнее, подбор гаммы проведем с помощью более быстрой кросс валидации.

In [ ]:
kf = KFold(n_splits=5)

gamma_scores = dict()
for gamma in tqdm(gammas):
    list_of_scores = []
    for train_index, test_index in tqdm(kf.split(X_train), 
                                        leave=False):
        x_train, x_test = X_train[train_index], X_train[test_index]
        y_lr_train, y_lr_test = y_train[train_index], y_train[test_index]

        model = LogisticRegression(penalty='l1', C=2/gamma, solver='liblinear',
                                   fit_intercept=False, random_state=0)
        model.fit(x_train, y_lr_train)
        list_of_scores.append(model.score(x_test, y_lr_test))
        
    gamma_scores[gamma] = np.mean(list_of_scores)


In [ ]:
best_gamma = sorted(gamma_scores, 
                    key=lambda x: gamma_scores[x], reverse=True)[0]
print("Оптимальное значение гамма:", best_gamma)
model = LogisticRegression(penalty='l1', C=2/best_gamma, solver='liblinear',
                                   fit_intercept=False, random_state=0)
model.fit(X_train, y_train)
print('Точность: {}'.format(model.score(X_test, y_test)))

В итоге, обе регуляризации дали почти те же результаты. В качестве представителя метода выберем l2 регуляризацию с $\gamma = 10$. Перейдем к сравнению методов.

In [ ]:
LR_best_model = LogisticRegression(penalty='l2', C=2/10, fit_intercept=False, 
                           random_state=0, solver='saga')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
LR_best_model.fit(X_train, y_train)
LR_best_report= classification_report(LR_best_model.predict(X_test), y_test)
print("Logistic regression with l2 regularization with gamma = 10\n", LR_best_report)

## Выводы

По итогам подбора гиперпараметров и регуляризации в финал вышли три модели: KNN c учетом 9 соседей с равными весами, перцептрон с двумя скрытыми слоями, размерности 64, и логистическая регрессия с l2 регуляризацией и параметром регуляризации $\gamma = 10$. Сравним их с помощью Cross validation.

Вспомним, что задача стоит в детекции злокачественной опухоли, а значит самое главное это увеличить True Negative Rate (TNR), то есть цель - это сделать так, чтобы все злокачественные опухоли были обнаружены. Будем сравнивать модели по точности и по TNR.

In [ ]:
#loo = LeaveOneOut()
n_splits=5
n_repeats=5
rkf = RepeatedKFold(n_splits=n_splits, n_repeats=n_repeats)
number_of_batch = n_splits*n_repeats
models = [KNN_best_model, Perceptron_best_model, LR_best_model]
          
model_scores = dict()
tn_scores = dict()
fp_scores=dict()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
for model in tqdm(models):
    list_of_scores = []
    list_tn = []
    list_fp = []
    for train_index, test_index in tqdm(rkf.split(X_train), 
                                        total=number_of_batch, leave=False):
        x_train, x_test = X_train[train_index], X_train[test_index]
        y_fight_train, y_fight_test = y_train[train_index], y_train[test_index]

        model.fit(x_train, y_fight_train)
        list_of_scores.append(model.score(x_test, y_fight_test))
        
        tn, fp, fn, tp = confusion_matrix(KNN_best_model.predict(X_test), y_test).ravel()
        list_tn.append(tn)
        list_fp.append(fp)
        
    model_scores[model] = np.mean(list_of_scores)
    tn_scores[model] = np.mean(list_tn)
    fp_scores[model] = np.mean(list_fp)

In [ ]:
accuracy = pd.DataFrame.from_dict(model_scores, orient='index')
FP = pd.DataFrame.from_dict(fp_scores, orient='index')
TN = pd.DataFrame.from_dict(tn_scores, orient='index')

metr = pd.concat([accuracy, TN, FP], axis = 1, join='inner')
metr.index = ['KNN', 'MLP', 'LR']
metr.columns = ['Accuracy', 'TN', 'FP']

TNR = metr['TN']/(metr['TN']+metr['FP'])
metr = pd.concat([metr, TNR], axis = 1, join='inner')
metr = metr.drop(labels=['TN', 'FP'], axis = 1)
metr.columns = ['Accuracy', 'TNR']

metr

Получили, что показателим точности и True Negative Rate для всех трех моделей очень близки к 1 и практически одинаковы. Логистическая регрессия показывает чуть большую точность, а KNN чуть более высокий TNR. Хотя результаты и получились очень близкими. 

Тут стоит вспомнить, что даже по одному признаку мы провели классификацию с точностью порядка $0,9$, что говорит о высокой предсказательной способности датасета, что и подтвердилось в ходе экспериментов.

Из недочетов стандартных библиотек было замечено, что в Multi Layer Perceptron не реализован dropout, что делает регуляризацию в рамках sklearn нереализуемой.

Также хочется отметить высокий перформанс у модели KNN и довольно низкую скорость обучения перцептрона, хотя точность получилась почти такая же. Их этого можно сделать вывод, что в данном датасете наиболее применимы простые модели с небольшим количеством гиперпараметров и высоким перформансом.